![STScI Logo](../../../_static/stsci_header.png)

<a id="titleEphemeris"></a>
# Getting HST Ephemeris and Related Data for an Observation
This notebook walks through how to get various information about where HST is, and where it is pointing, in relation to Earth and the Sun, for a given observation. 


## Table of Contents
[Introduction](#intro_ID) <br>
[Imports](#imports) <br>
[Download the Data](#download) <br>
[1. Earth Limb Angle](#earth-limb-angle) <br>
[2. HST Lattude & Longitude During Observation](#hst-lattude--longitude-during-observation) <br>
[3. Solar Time, Solar Altitude, and Solar Cycles](#solar-time-solar-altitude-and-solar-cycles) <br>
[About this Notebook](#about)

<a id="intro_ID"></a>
## Introduction

### Worked Examples for an ACS/SBC Observation 

This notebook walks through how to get various information about where HST is, and where it is pointing, in relation to Earth and the Sun, for a given observation. 

The methods laid out in this notebook do not necessarily represent the "official" way that one would extract these quantities from the full NASA engineering data. However, interacting with the engineering data can be non-trivial, and there is often limited documentation.

Therefore, this notebook aims to provide ways to calculate various ephemeris (and related) quantities in ways that are reliable, whilst also still being relatively straightforward.

Some of the examples laid out in this notebook use information available in the FITS header, and some use other data available online. To run this notebook, you will require `astropy`, `astroquery`, `pandas`, `pvlib`, and `ppigrf` to be installed (along with their dependences).

In this example, we will be using an observation from the ACS/SBC - namely `jec0c6aeq_flc.fits`. It's an observation of part of nearby galaxy NGC7793, looking at a bunch of young stars and star-forming regions. This file will be downloaded from MAST, and put in a directory your current working directory (in general, this will automatically be the same directory as the notebook itself, under standard notebook kernel server settings).

Note that the methods demonstrated in this notebook *require* you to be working with a file associated with an individual expoure (ie, a `raw`, `flt`, or `flc` file). This is because a drizzled file, or other combined data product, will consist of data from different observations, where the telescope, Earth, and Sun will have been in different relative positions.

This notebook reflects the methods used to obtain various observaitonal parameters used in ACS ISR 2025-05.

<a id="imports"></a>
## Imports

In [1]:
# Required imports all here
import os
import requests
from io import StringIO
import numpy as np
import pandas as pd
import astropy.io.fits
import astropy.wcs
import astropy.time
import astropy.convolution
import astropy.units as u
import astropy.coordinates
import astroquery.mast
import skyfield.api
import ppigrf
import pvlib

# Report to user where data folder will be palced
data_dir = os.path.join(os.getcwd())
print('Data for this notebook will be downloaded to:')
print(data_dir)

Data for this notebook will be downloaded to:
/home/runner/work/hst_notebooks/hst_notebooks/notebooks/ACS/hst_orbits_ephem


<a id="download"></a>
## Download the Data

Before we get going, we'll download the obesrvation we want from MAST, and read in its headers.

In [2]:
# Create folder for holding downloaded files, etc

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

# Download our exmaple observation
flt_filename = 'jec0c6aeq_flt.fits'
flt_path = os.path.join(data_dir, flt_filename)
mast_msg = astroquery.mast.Observations.download_file('mast:HST/product/'+flt_filename,
                                                      local_path=flt_path,
                                                      cache=False)
print(mast_msg[0])

# Read in headers for our observation; we will require info from the primary and image headers
flt_hdr_0 = astropy.io.fits.getheader(flt_path, ext=0)
flt_hdr_1 = astropy.io.fits.getheader(flt_path, ext=1)

 [Done]


COMPLETE


<a id="earth-limb-angle"></a>
## 1. Earth Limb Angle

Earth Limb Angle (ELA) is the angle between the locaiton where the telescope is pointing, and the limb of the Earth. As ELA gets smaller, the atmosphere can have an increasing effect on HST observations in certain wavelenghts (especially the IR and UV). So knowing the ELA is worthwhile for understanding backgrounds, etc.

Establishing ELA for a given observation is a somewhat involved process, so we will deal with it first.

To get the ELA, we need to get the jitter file (ie, `_jit.fits` file for this observation. If this observation was not part of an association, then the `jit` file should have the same rootname (ie, `jec0c6aeq`) as the `flt` file. However, if this oabservation was part of an assocation, then the `jit` file will have the same rootname as the associaiton. So we will check for both

In [3]:
# First, check if jit file exists with same rootname as flt
jit_filename = flt_filename.replace('_flt.fits', '_jit.fits')
jit_path = os.path.join(data_dir, jit_filename)
mast_msg = astroquery.mast.Observations.download_file('mast:HST/product/'+jit_filename,
                                                      local_path=jit_path,
                                                      cache=False)
if os.path.exists(jit_path) and (mast_msg[0] != 'ERROR'):
    jit_data = astropy.io.fits.getdata(jit_path)

# Otherwise, get the jit file by finding the association of this observation
elif 'HTTPError: 404 Client Error: Not Found for url' in mast_msg[1]:
    asn_id = flt_hdr_0['ASN_ID'].lower()
    jit_filename = asn_id+'_jit.fits'
    jit_path = os.path.join(data_dir, jit_filename)
    mast_msg = astroquery.mast.Observations.download_file('mast:HST/product/'+jit_filename,
                                                          local_path=jit_path,
                                                          cache=False)
    jit_data = astropy.io.fits.getdata(jit_path)
print(mast_msg[0])

 [Done]


COMPLETE


Now we have the `jit` file read it, we grab the ELA data from it. This consists of an array of ELA values, as the ELA changes continually throughout the exposure.

In [4]:
# Get ELA data from JIT file
ela_arr = np.array(jit_data['LimbAng'])

# Check that ELA data is good (can be an empty array for certain engineering data, etc)
try:
    ela_min = np.min(ela_arr)
except ValueError:
    print('ELA array is empty')

# Assuming ELA data is good, get some summary statistics
ela_min = np.min(ela_arr)
ela_max = np.max(ela_arr)
ela_mean = np.mean(ela_arr)
print(f'ELA ranges from {ela_min:.1f} deg to {ela_max:.1f} deg, with a mean of {ela_mean:.1f} deg')

ELA ranges from 39.6 deg to 52.9 deg, with a mean of 46.3 deg


<a id="hst-lattude--longitude-during-observation"></a>
## 2. HST Lattude & Longitude During Observation

If you want to know the position of HST over the Earth during a given observation, you have to account for the fact that HST is continually moving in three dimensions around the Earth, which is continually moving in three dimensions around the Sun (thanks, Copernicus). This is a whole bunch of 3D ephemeris maths we'd rather not do by hand.

Fortunately, we don't have to! The United States Space Command (USSPACECOM) provides public orbital information, which can be converted into the necessary ephemeres using the `skyfield` package.

First, we have to know exactly when the observation was taken, which we can get from the header. We then use that to construct an `astropy.time.Time` object.

In [5]:
# Get observation date and time fromt the header
date_string = flt_hdr_0['DATE-OBS']
time_string = flt_hdr_0['TIME-OBS']

# Convert into an astropy Time object
datetime_obs_start = astropy.time.Time(date_string+'T'+time_string, 
                                       format='isot', 
                                       scale='utc')

Now we use the datetime, and the exposure time, to find the mid-point time of the exposure, and construct an array sampling the whole duration of the observation. We also compute the Modified Julian Date (MJD) of the mid-point of the exposure, as this will be a useful number later

In [6]:
# Use duration of exposure to find time in middle of exposure
exp_time = flt_hdr_0['EXPTIME'] * u.second
datetime_mid_delta = exp_time * 0.5
datetime_obs_mid = datetime_obs_start + datetime_mid_delta

# Make array densely sampling time throughot the observation (1000 points is a good arbitrary dense sampling)
datetime_obs_arr = np.linspace(datetime_obs_start, 
                               (datetime_obs_start + exp_time), 
                               num=1000)

# Record date and time info
mjd_obs_mid = astropy.time.Time(datetime_obs_mid).mjd

# Report summary of observation time
print(f'Exposure started at {datetime_obs_arr[0]}, and ended at {datetime_obs_arr[-1]}')
print(f'The mid-point time of the exposure was at MJD of {mjd_obs_mid:.5f}')

Exposure started at 2022-04-25T03:18:51.000, and ended at 2022-04-25T03:22:41.000
The mid-point time of the exposure was at MJD of 59694.13942


With all this date & time info ready, we can now use the USSPACECOM data to work out exactly where HST was during this exposure.

The data provided by USSPACECOM comes in the form of 'Two Line Elements', or TLEs. A table of TLEs for HST complete up until early November 2025 is provided with this notebook, called `HST_TLEs.dat`. If you need data that extend to later dates, the official USSPACECOM TLE tables can be retrieved from `https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/20580/format/tle`.

With that data in-hand, we can now use the skyfield package to read in the TLE table.

In [7]:
# Set up a skyfield timescale object, and prepare a storage variable for results
timescale = skyfield.api.load.timescale()
tle_list = []

# Read in TLE file
tle_file = 'HST_TLEs.dat'
with open(tle_file, 'r') as f:
    lines = [line.strip() for line in f if line.strip()]

# Loop over lines of TLE file, processing them with skyfield
for i in range(0, len(lines), 2):
    line1 = lines[i]
    line2 = lines[i+1]
    sat = skyfield.api.EarthSatellite(line1, line2, 'HST', timescale)
    tle_list.append(sat)

Having read in our TLE data, we now search through the TLEs to find the one that most closely matches the date of our observation, and use the `skyfield` package to compute the corresponding positional data. Note that the Skyfield package uses Julia dates, not _modifed_ Julian dates, for this.

In [8]:
# Set up a skyfield timescale object, and prepare a storage variable for results
timescale = skyfield.api.load.timescale()
hst_orbit_coords = []

# Get Julian date (which skyfield uses) 
datetime_obs_mid_jd = datetime_obs_mid.jd
time_sf = timescale.tt_jd(datetime_obs_mid_jd)

# Select the TLE whose epoch is closest to the requested date
closest_sat = min(tle_list, key=lambda s: abs(s.epoch.tt - datetime_obs_mid_jd))
subpoint = closest_sat.at(time_sf).subpoint()

# Get coordinates, and put them into an astropy coordinates object
hst_location_lat = subpoint.latitude.degrees * astropy.units.deg
hst_location_lon = subpoint.longitude.degrees * astropy.units.deg
hst_location_height = subpoint.elevation.km * astropy.units.km
hst_location = astropy.coordinates.EarthLocation(lat=hst_location_lat, 
                                                 lon=hst_location_lon, 
                                                 height=hst_location_height)

# Report location to user
print(f'During exposure mid-point, HST was above lat = {hst_location_lat:.2f}, lon = {hst_location_lon:.2f}, at height of {hst_location_height:.0f}')

During exposure mid-point, HST was above lat = -27.87 deg, lon = 154.90 deg, at height of 541 km


As an example use-case of this, we can compute the *rough* distance from the South Atlantic Anomaly (SAA). Note that the SAA is not symmetric, and that any proper SAA calculations should take account of its contours. And the central point of the SAA changes. So this for illustrative purposes only.

In [9]:
# Give approxiamte coords of centre of SAA
saa_lat, saa_lon = -25.603*u.deg, -40.287*u.deg # Approximate coords, but good enough
saa_location = astropy.coordinates.EarthLocation.from_geodetic(lat=saa_lat, lon=saa_lon)

# Compute separation between SAA and HST, in degrees, and report to user
saa_itrs = astropy.coordinates.SkyCoord(saa_location.get_itrs())
hst_itrs = astropy.coordinates.SkyCoord(hst_location.get_itrs())
saa_distance = saa_itrs.separation(hst_itrs)
print(f'HST was {saa_distance:.1f} from the approximate middle of the SAA')

HST was 124.8 deg from the approximate middle of the SAA


Another use for knowing HST's position is to compute the geomagnetic field strength at that location, at the time of obesrvaiton. The strength of the geomagnetic field can affect UV airglow, and impact HST's electronics (most notably over the SAA, where the field is especially weak). To to this, we can use the 14th version of the International Geomagnetic Reference Field (IGRF-14; [Alken et al., 2021](http://dx.doi.org/10.1186/s40623-020-01288-x)), as implemented by the `ppigrf` package.

In [10]:

# Use pyigrf to get the geomagnetic field strength components at the location of HST during the observation
igrf_components = ppigrf.igrf(hst_location_lon, 
                              hst_location_lat, 
                              hst_location_height,
                              datetime_obs_mid.datetime)

# pyigrf gives the eastward, northward, and upward components of the field; add them in quadrature to get total field stegnth
igrf_total = np.sqrt(np.sum(np.array(igrf_components)**2.0)) * u.nT
print(f'Geomagnetic field strength during mid-point of observation was {igrf_total:.0f}')

Geomagnetic field strength during mid-point of observation was 40786 nT


<a id="solar-time-solar-altitude-and-solar-cycles"></a>
## 3. Solar Time, Solar Altitude, and Solar Cycles

Now we can do a bunch of Solar maths regarding where, and when, HST peformed this exposure.

To start off, let's calculate Solar time. This is the time system where the Sun is overhead at 12 noon at whatever your location happens to be.  This can be relevant in filters where the atmosphere gets excited by Solar radiation during the course of the day, resulting in increased airglow at dusk vs dawn (even though comparable dust and dawn times may have the same Solar altitude).

We get the Sun's location using the package `pvlib`, which is designed for Solar maths. Then we compute the 'equation of time' for HST during the exposure. The 'equation of time' is the difference between apparent Solar time and mean Solar time for a given time & place.

In [11]:
# First, find the Sun's location 
sun_location = pvlib.location.Location(hst_location.lat.value, hst_location.lon.value).get_solarposition(datetime_obs_mid.datetime)

# Find the equation of time for HST, and thence the longitude correction required to get actual Solar time from HST's position
hst_eot = (sun_location.equation_of_time.array[0] * u.minute).to(u.hour)
hst_lon_corr = hst_location.lon / 15.0 * u.hour/u.deg

# Use the equation fof time to calculate solar time at HST's location, as an astropy.time.Time object
datetime_obs_mid_solar = datetime_obs_mid + hst_eot + hst_lon_corr

# Get out the solar time as an actual number, hours
hst_solar_time_hr = datetime_obs_mid_solar.datetime.hour + (datetime_obs_mid_solar.datetime.minute / 60.0) + (datetime_obs_mid_solar.datetime.second / 3600.0)
print(f'Solar time for HST during exposure was {hst_solar_time_hr:05.2f} hours (in 24-hour notation)')

Solar time for HST during exposure was 13.71 hours (in 24-hour notation)


Now, we'll calculate the altitude and azimuth of the Sun, from the position of HST. Turns out that `astropy.coordinates` has a built-in function for this, assuming you construct the right coordinate frame first.

Note that the altitude and azimuth will be for the plane of HST *tangent* to the surface of the Earth, at the location of HST. Therefore, the Earth limb will be at a negative altitude, due to the height of HST's orbit above the surface.

In [12]:
# Use astropy coordinates to construct an alt-az reference frame object at the place and time of the exposure
altaz_frame = astropy.coordinates.AltAz(obstime=datetime_obs_mid, location=hst_location)

# Now use astropy function to compute Solar position in this reference frame
sun_altaz = astropy.coordinates.get_sun(datetime_obs_mid).transform_to(altaz_frame)
print(f'Sun\'s position during exposure was altitude = {sun_altaz.alt:.2f}, azimuth = {sun_altaz.az:.2f}')

Sun's position during exposure was altitude = 42.06 deg, azimuth = 325.50 deg


Now to finish with a couple of easy calculations. Firstly, lets find out how many sunspots were on the Sun, during our observation. The atmosphere extends to higher altitudes during increased Solar activity (which is traced by more sunspots), so this can once again be a relevant quantitity for airglow.

Historical daily sunspot numbers are provided by the Royal Observatory of Belgium’s Sunspot Index and Long-term Solar Observations (SILSO) database.

In [13]:
# First, state the dates of of the Solar maxima  the once before HST launch
silso_url = 'http://www.sidc.be/silso/DATA/SN_d_tot_V2.0.csv'
silso_query = requests.get(silso_url)
silso_cols = ['year', 'month', 'day', 'decimal_date', 'daily_sunspot_number',
              'std_dev', 'observations', 'definitive_provisional']
silso_frame = pd.read_csv(StringIO(silso_query.text), sep=';', header=None, names=silso_cols)

# Extract SILSO sunspot counts in date range of interest
silso_frame['datetime'] = pd.to_datetime(silso_frame[['year', 'month', 'day']])
silso_date_start = (datetime_obs_mid - (3 * u.day)).strftime('%Y-%m-%d')
silso_date_end = (datetime_obs_mid + (3 * u.day)).strftime('%Y-%m-%d')
silso_date_where = np.where((silso_frame['datetime'] >= silso_date_start) & (silso_frame['datetime'] <= silso_date_end))

# Find average number of sunspots for week centred on observation date, and report
silso_spots_mean = np.mean(silso_frame.loc[silso_date_where, 'daily_sunspot_number'])
print(f'Average number of sunspots was {silso_spots_mean:.1f} during week of observation ')

Average number of sunspots was 126.3 during week of observation 


Lastly, let's convert the ra and dec of our HST observations to ecliptic coordinates. This can be useful if we're worrdied about the potential for zodiacal light to affect our observations. For this calculation, we're going to grab the centre coord of the image in our `flt` file.

In [14]:
# Get ra and dec from header; here we just use the WCS reference coord (you may want to be more precise)
ra = flt_hdr_1['CRVAL1A'] * u.deg
dec = flt_hdr_1['CRVAL2A'] * u.deg

# Convert coordinates to ecliptic frame, and report to user
icrs_coords = astropy.coordinates.SkyCoord(ra=ra, dec=dec, frame='icrs')
ecliptic_coords = icrs_coords.transform_to(astropy.coordinates.GeocentricTrueEcliptic)
print(f'Reference pixel of observation has ecliptic coords of lat = {ecliptic_coords.lat:.2f}, lon = {ecliptic_coords.lon:.2f}')

Reference pixel of observation has ecliptic coords of lat = -29.42 deg, lon = 345.26 deg


<a id="about"></a>
## About this Notebook
**Author:** Chris Clark (ESA/AURA, Space Telescope Science Institute; `cclark@stsci.edu`) <br>
**Updated On:** 12/17/2025

If you use `astropy` or `pvlib` for published research, please cite the authors. Follow these links for more information about citing `astropy` or `pvlib`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `pvlib`](https://pvlib-python.readthedocs.io/en/stable/index.html#citing-pvlib-python)

### For more help:

More details may be found on the [ACS website](http://www.stsci.edu/hst/instrumentation/acs) and in the [ACS Instrument](https://hst-docs.stsci.edu/display/ACSIHB) and [Data Handbooks](https://hst-docs.stsci.edu/acsdhb).

Please visit the [HST Help Desk](http://hsthelp.stsci.edu). Through the help desk portal, you can explore the HST Knowledge Base and request additional help from experts.

<hr>

[Top of Page](#titleEphemeris)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 
<br></br>
<br></br>

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>